In [22]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import numpy as np

from solution import *

In [24]:
n_colors = 10

colors = px.colors.sample_colorscale("turbo", [n/(n_colors -1) for n in range(n_colors)])

In [27]:
s = Solution()

In [29]:
s.graph[0]

AtlasView({1: {'orient': <Orient.NORTH: 1>, 'space_rel': <SpatialRel.ADJACENT: 1>}, 2: {'orient': <Orient.EAST: 3>, 'space_rel': <SpatialRel.ADJACENT: 1>}, 3: {'orient': <Orient.SOUTH: 2>, 'space_rel': <SpatialRel.CONTAINS: 3>}})

In [60]:
for ix, node in enumerate(s.graph.nodes):
    print(f"node {ix}")
    for face in s.graph.nodes[ix]["props"].faces.face_list:
        print(face.name, face.sols)

node 0
faceN [13]
faceS [10]
faceE [13]
faceW [10]
faceT [3]
faceB [0]
node 1
faceN [49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
faceS [49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
faceE [49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
faceW [49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
faceT [49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18,

In [55]:
def format_face_sols(face:Face):
    mid_domain = 25
    return np.array(face.sols), np.repeat(mid_domain, len(face.sols))


In [56]:
def create_node_data(face_list:[Face]):
    node_data = {}
    for face in face_list:
        x, y = format_face_sols(face)

        if face.axis == Axes.X:
            node_data[face.name] = [x, y]
        elif face.axis == Axes.Y:
            node_data[face.name] = [y, x]
        else: 
            pass
    return node_data



In [65]:
node_viz_data = []

for ix, node in enumerate(s.graph.nodes):
        node_viz_data.append(create_node_data(s.graph.nodes[ix]["props"].faces.face_list))

# node_viz_data

In [78]:
node_names = [str(ix) for ix in s.graph.nodes]
fig = sp.make_subplots(rows=1, cols=len(node_viz_data), subplot_titles=node_names)

for ix, faces in enumerate(node_viz_data):
    showlegendbool = True if ix == 0 else False
    for iy, (face_name, face_coords) in enumerate(faces.items()):
        fig.add_trace(go.Scatter(x=face_coords[0], y=face_coords[1], name=face_name, marker_color=colors[iy], showlegend=showlegendbool), row=1, col=ix+1)


fig.show()